In [1]:
import numpy as np
import rospy
import struct
import math
import time
import pandas as pd
from scipy.spatial.transform import Rotation as R
import tf

from sensor_msgs import point_cloud2
from sensor_msgs.msg import PointCloud2, PointField
from std_msgs.msg import Header
from tf2_sensor_msgs.tf2_sensor_msgs import do_transform_cloud
import geometry_msgs.msg

In [2]:
class PointCloud_Shape:
    def __init__(self, points, pose):
        self.br = tf.TransformBroadcaster()
        self.header = Header()
        self.pointcloud_publisher = rospy.Publisher('test/pcl', PointCloud2, queue_size=10)
        
        self.points = points
        self.pose = pose

    def make_pcl(self, points):
        header = self.header
        header.stamp = rospy.Time.now()
        header.frame_id = "sensor_bundle"

        # print(type(points), points)

        fields = [PointField('x', 0, PointField.FLOAT32,1),
                  PointField('y', 4, PointField.FLOAT32,1),
                  PointField('z', 8, PointField.FLOAT32,1),
                  PointField('intensity', 12, PointField.FLOAT32,1)]

        pcl = point_cloud2.create_cloud(header, fields, points)
        return pcl

    def set_tf(self):
        self.br.sendTransform([0, 0, 0],
                              [0, 0, 0, 1],
                              rospy.Time.now(),
                              "sensor_bundle",
                              "world")

    def make_point(self, thresh_hold=0.5):

        sensor_num = 19

        for point_num in range(self.points.shape[1]):
            sensor_to_point_cloud = []

            for sensor in range(sensor_num):

                if self.points[sensor+sensor_num, point_num] > thresh_hold:

                    data = self.points[sensor, point_num]
                    
                    if  0.75 >= data and data >= 0.001:
                        intensity = 0.1
                    else:
                        intensity = 0.9
                        
                    Full_quarternion = self.pose[sensor*7:(sensor+1)*7, point_num]
                    point = [0.0, 0.0, data]

                    translation = Full_quarternion[:3]
                    quarternion = Full_quarternion[3:].tolist()

                    rot_matrix = R.as_dcm(R.from_quat(quarternion))

                    world_point = (np.dot(rot_matrix, point) + translation).tolist()
                    world_point.append(intensity) 

                    sensor_to_point_cloud.append(world_point)

                pcl = self.make_pcl(sensor_to_point_cloud)
                self.pointcloud_publisher.publish(pcl)
                time.sleep(0.001)

            print("Making", point_num, self.points.shape[1])
        
        print("Making Done")

In [3]:
def load_data(mode='test'):
    points = None
    pose = None

    if mode == 'test':
        for i in range(1):
            path1 = '/home/jee/work_space/data_folder/Sensor_learning/data/hexagon/test_3/output_Fold_%d.csv'%(i+1)
            path2 = '/home/jee/work_space/data_folder/Sensor_learning/data/hexagon/test_3/pose_Fold_%d.csv'%(i+1)
            temp_points = np.array(pd.read_csv(path1, sep=",", header=None))
            temp_pose = np.array(pd.read_csv(path2, sep=",", header=None))

            points = merge_data(points, temp_points, axis=1)
            pose = merge_data(pose, temp_pose, axis=1)

        points = points.T


    elif mode == 'predict':
        for i in range(1):
            path2 = '/home/jee/work_space/data_folder/Sensor_learning/data/hexagon/test_3/pose_Fold_%d.csv'%(i+1)
            temp_pose = np.array(pd.read_csv(path2, sep=",", header=None))

            pose = merge_data(pose, temp_pose, axis=1)

        path1 = '/home/jee/work_space/data_folder/Sensor_learning/data/hexagon/result/predict_Fold_1_3.csv'
        points = np.array(pd.read_csv(path1, sep=",", header=None))

    pose = pose.T

    print('Data Load Done')
    print(points.shape)
    print(pose.shape)

    return points, pose

def merge_data(total_data, add_data, axis=0):
    total = total_data

    if type(total)==type(None):
        total = add_data
#         print('merge ok')
    else:
        total = np.concatenate((total, add_data),axis)
    
    total = pd.DataFrame(total)
    total = np.array(total)

    return total


In [4]:
rospy.init_node('make_point_cloud')
print("start")

start


In [5]:
points, pose = load_data(mode='predict')

Data Load Done
(76, 203391)
(266, 203391)


In [6]:
point_cloud_vrep = PointCloud_Shape(points, pose)
while not rospy.is_shutdown():
    try:
        # point_cloud_vrep.set_tf()
        point_cloud_vrep.make_point(thresh_hold=0.7)

    except rospy.ROSInterruptException:
        pass

('Making', 0, 203391)
('Making', 1, 203391)
('Making', 2, 203391)
('Making', 3, 203391)
('Making', 4, 203391)
('Making', 5, 203391)
('Making', 6, 203391)
('Making', 7, 203391)
('Making', 8, 203391)
('Making', 9, 203391)
('Making', 10, 203391)
('Making', 11, 203391)
('Making', 12, 203391)
('Making', 13, 203391)
('Making', 14, 203391)
('Making', 15, 203391)
('Making', 16, 203391)
('Making', 17, 203391)
('Making', 18, 203391)
('Making', 19, 203391)
('Making', 20, 203391)
('Making', 21, 203391)
('Making', 22, 203391)
('Making', 23, 203391)
('Making', 24, 203391)
('Making', 25, 203391)
('Making', 26, 203391)
('Making', 27, 203391)
('Making', 28, 203391)
('Making', 29, 203391)
('Making', 30, 203391)
('Making', 31, 203391)
('Making', 32, 203391)
('Making', 33, 203391)
('Making', 34, 203391)
('Making', 35, 203391)
('Making', 36, 203391)
('Making', 37, 203391)
('Making', 38, 203391)
('Making', 39, 203391)
('Making', 40, 203391)
('Making', 41, 203391)
('Making', 42, 203391)
('Making', 43, 203391